In [ ]:
import os
import time
import random
from collections import Counter

import numpy as np
import pandas as pd
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torch.utils.data import DataLoader, Subset
from torch.optim.lr_scheduler import ReduceLROnPlateau

import torchvision
from torchvision import datasets, transforms
from torchvision.models import densenet121, DenseNet121_Weights  # DenseNet-121

from sklearn.model_selection import KFold
import optuna

import matplotlib.pyplot as plt

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

def seed_everything(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

seed_everything(42)

In [ ]:
#Image preprocessing
train_transform = transforms.Compose([transforms.Resize((456, 456)),
                                      transforms.RandomHorizontalFlip(), 
                                      transforms.RandomVerticalFlip(),
                                      transforms.ToTensor(), 
                                      transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) 
                                     ])

valid_test_transform = transforms.Compose([transforms.Resize((456, 456)), 
                                     transforms.ToTensor(),
                                     transforms.Normalize(
                                         mean=[0.485, 0.456, 0.406], 
                                         std=[0.229, 0.224, 0.225])
                                    ])

In [ ]:
# Dataset folder path
dataset_dir = 'set your own path'
all_data_path = os.path.join(dataset_dir, 'train')
valid_path = os.path.join(dataset_dir, 'valid')
test_path = os.path.join(dataset_dir, 'test')

all_dataset = datasets.ImageFolder(all_data_path, train_transform) 
valid_dataset = datasets.ImageFolder(valid_path, valid_test_transform) 
test_dataset = datasets.ImageFolder(test_path, valid_test_transform)

In [ ]:
# class
class_names = all_dataset.classes
n_class = len(class_names)
all_dataset.class_to_idx
idx_to_labels = {y:x for x,y in all_dataset.class_to_idx.items()}

In [ ]:
#cross-validation to split the training dataset
kf = KFold(n_splits=5, shuffle=True, random_state=42)
folds = []

for train_idx, val_idx in kf.split(np.arange(len(all_dataset))):
    folds.append((train_idx, val_idx))

In [ ]:
# DataLoader
batch_size = 8

train_loaders = []
val_loaders = []

for fold, (train_idx, val_idx) in enumerate(folds):
    train_subset = Subset(all_dataset, train_idx)
    val_subset = Subset(all_dataset, val_idx)
    
    train_loader = DataLoader(train_subset, batch_size=batch_size, shuffle=True, num_workers=4)
    val_loader = DataLoader(val_subset, batch_size=batch_size, shuffle=False, num_workers=4)
    
    train_loaders.append(train_loader)
    val_loaders.append(val_loader)

In [ ]:
#focal_loss
class FocalLoss(nn.Module):
    def __init__(self, alpha=0.25, gamma=2.0, reduction='mean'):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.reduction = reduction

    def forward(self, inputs, targets):
        # inputs: raw logits of shape [B, C]
        # targets: ground-truth labels of shape [B]
        log_probs = F.log_softmax(inputs, dim=1)
        probs = torch.exp(log_probs)
        targets_one_hot = F.one_hot(targets, num_classes=inputs.size(1)).float()

        pt = (probs * targets_one_hot).sum(dim=1)  # shape: [B]
        log_pt = (log_probs * targets_one_hot).sum(dim=1)

        alpha_t = self.alpha * targets_one_hot + (1 - self.alpha) * (1 - targets_one_hot)
        alpha_t = alpha_t.sum(dim=1)  # shape: [B]

        loss = -alpha_t * (1 - pt) ** self.gamma * log_pt

        if self.reduction == 'mean':
            return loss.mean()
        elif self.reduction == 'sum':
            return loss.sum()
        else:
            return loss

In [ ]:
# Optuna to find hyperparameters
def objective(trial):
    learning_rate = trial.suggest_float('lr', 1e-5, 1e-2, log=True) 
    batch_size = trial.suggest_int('batch_size', 4, 32)  
    optimizer_choice = trial.suggest_categorical('optimizer', ['Adam', 'SGD']) 
    weight_decay = trial.suggest_float('weight_decay', 1e-6, 1e-3, log=True)  
    dropout_rate = trial.suggest_float('dropout', 0.3, 0.7)  
    epochs = trial.suggest_int('epochs', 5, 50)  

    #Focal Loss parameters
    alpha = trial.suggest_float('alpha', 0.1, 0.5)
    gamma = trial.suggest_float('gamma', 1.0, 5.0)
    
    val_accs = []  

    num_classes = 2 

    #Class weighting parameters
    class_weights = [total_samples / (num_classes * class_counts[i]) for i in range(num_classes)]
    class_weights_tensor = torch.tensor(class_weights, dtype=torch.float32).to(device)
    
    for fold in range(5):
        train_loader = train_loaders[fold]
        val_loader = val_loaders[fold]

        model = models.densenet121(weights=models.DenseNet121_Weights.IMAGENET1K_V1)
        
        for param in model.parameters():
            param.requires_grad = False
        
        in_features = model.classifier.in_features
        model.classifier = nn.Sequential(
            nn.Dropout(p=dropout_rate),
            nn.Linear(in_features, num_classes)
        )
        
        model = model.to(device)  

        if optimizer_choice == 'Adam':
            optimizer = optim.Adam(model.classifier.parameters(), lr=learning_rate, weight_decay=weight_decay)
        else:
            optimizer = optim.SGD(model.classifier.parameters(), lr=learning_rate, weight_decay=weight_decay, momentum=0.9)

        criterion = nn.CrossEntropyLoss()

        for epoch in range(10):  
            model.train()
            running_loss = 0.0
            for inputs, labels in train_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                
                optimizer.zero_grad()

                outputs = model(inputs)
                loss = criterion(outputs, labels)

                loss.backward()
                optimizer.step()

                running_loss += loss.item()

        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        val_acc = correct / total
        val_accs.append(val_acc)
        
    return np.mean(val_accs)

study = optuna.create_study(direction='maximize')  
study.optimize(objective, n_trials=10)

print(f"Best hyperparameters: {study.best_params}")

In [ ]:
#save hyperparameters
import joblib
joblib.dump(study, "optuna_original.pkl")